In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import altair as alt
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import HBox, VBox, Layout
import warnings
warnings.filterwarnings("ignore")

# dati ISTAT
data = pd.read_csv("DCSP_COLTIVAZIONI_22032024191007716.csv")

# Rimozione spazi bianchi dopo le stringhe della colonna 'Tipo dato'
data['Tipo dato'] = data['Tipo dato'].str.rstrip()

# Filtro 'Tipo dato' di interesse
data_filtered = data[data["Tipo dato"].isin(['superficie totale - ettari', 'superficie totale - are', 
                                           'superficie in produzione - ettari', 'produzione totale - quintali', 
                                           'produzione totale - migliaia di quintali', 'produzione raccolta - quintali'])]

# Selezione del territorio di interesse
Territorio = 'Italia'

# Selezione degli attributi che si vogliono osservare
#attributo_1 = 'superficie totale - ettari'
#attributo_2 = 'produzione totale - quintali'
attributo_1 = 'produzione totale - quintali'
#attributo_2 = 'produzione raccolta - quintali'
#attributo_1 = 'superficie totale - ettari'
attributo_2 = 'superficie in produzione - ettari'

data_filtered2 = data_filtered
column_of_interest = ['Tipo dato', 'Tipo di coltivazione', 'TIME', 'Value', 'Territorio']
data_filtered3 = data_filtered2[column_of_interest]

In [2]:
import pandas as pd
import altair as alt
from ipywidgets import widgets, VBox, HBox

# Dark theme custom for Altair
dark_theme_custom = {
    'config': {
        'background': '#121212',  # Background color of the chart
        'title': {
            'color': '#E0E0E0',  # Title color
            'fontSize': 20
        },
        'axis': {
            'domainColor': '#B0B0B0',
            'gridColor': '#333333',
            'tickColor': '#B0B0B0',
            'titleColor': '#E0E0E0',
            'labelColor': '#E0E0E0',
            'labelFontSize': 12,  # Font size for axis labels
            'titleFontSize': 14   # Font size for axis titles
        },
        'legend': {
            'labelColor': '#E0E0E0',
            'titleColor': '#E0E0E0'
        },
        'mark': {
            'color': '#1f77b4'  # Mark color
        }
    }
}

# Register and apply the custom theme
alt.themes.register('dark_theme_custom', lambda: dark_theme_custom)
alt.themes.enable('dark_theme_custom')

# Sample DataFrame (replace with your actual DataFrame)
# data_filtered3 = ...

# Filtering Data
Territorio = 'Italia'
attributo_2 = 'produzione totale - quintali'
data_filtered4 = data_filtered3[data_filtered3['Tipo dato'] == attributo_2]
data_filtered5 = data_filtered4[data_filtered4['Territorio'] == Territorio]

data_filtered5.rename(columns={'Value': 'Valore (in quintali)'}, inplace=True)

# Calculate Percentage
total_by_year = data_filtered5.groupby('TIME')['Valore (in quintali)'].transform('sum')
data_filtered5['Percentage'] = (data_filtered5['Valore (in quintali)'] / total_by_year) * 100
data_filtered5['Percentage'] = data_filtered5['Percentage'].round(2)

# Sort and Filter Data
df_sorted = data_filtered5.sort_values(by='Percentage', ascending=False)
df_sorted_filtered = df_sorted[df_sorted['Percentage'] > 1.00]
df_sorted_filtered = df_sorted_filtered[df_sorted_filtered['TIME'] != 2024]

# Ensure df_sorted_filtered is not empty
if df_sorted_filtered.empty:
    raise ValueError("No data available after filtering. Please check your filters or data.")

# Dropdown options (list of TIME)
dropdown_options = sorted(df_sorted_filtered['TIME'].unique().tolist())

# Custom CSS style for the dropdown widget
custom_css = """
<style>
    .custom-dropdown .widget-dropdown {
        background-color: #121212;
        color: #E0E0E0;
        border: 1px solid #B0B0B0;
    }
    .custom-dropdown .widget-label {
        color: #E0E0E0;
    }
</style>
"""

# Initialize dropdown widget with dark theme
dropdown_category = widgets.Dropdown(
    options=dropdown_options,
    value=dropdown_options[0],
    description='Anno:',
    style={'description_width': 'initial'},  # Adjust the width of the description
    layout=widgets.Layout(width='200px'),
)

# Add custom CSS class to the dropdown widget
dropdown_category.add_class('custom-dropdown')

# Display custom CSS
display(widgets.HTML(custom_css))

# Function to update chart based on selected TIME
def update_chart1(TIME1):
    filtered_df = df_sorted_filtered[df_sorted_filtered['TIME'] == TIME1]
    chart = alt.Chart(filtered_df).mark_bar(color='#18E169').encode(
        x='Tipo di coltivazione',
        y='Valore (in quintali)',
        tooltip=['Tipo di coltivazione', 'Valore (in quintali)']
    ).properties(
        width=400,
        height=300,
        title='Produzione totale - quintali'
    )
    display(chart)

# Output widget for the chart
output1 = widgets.Output()

# Function to handle dropdown change
def on_dropdown_change(change):
    with output1:
        output1.clear_output()
        update_chart1(change['new'])

# Link the dropdown change to the handler
dropdown_category.observe(on_dropdown_change, names='value')

# Initial display
with output1:
    update_chart1(dropdown_category.value)

# Arrange the outputs (charts) horizontally using HBox layout
display(VBox([dropdown_category, output1]))


HTML(value='\n<style>\n    .custom-dropdown .widget-dropdown {\n        background-color: #121212;\n        co…